# Apriori Algorithm
Questo primo esercizio consiste nell'implementare l'algoritmo Apropri (usato per identificare itemset frequenti e generare regole di associazioni in un dataset. Per riuscire a fare questo, viene utilizzato il concetto del 'supporto' -> esso verifica quali itemset soddisfano una soglia di frequenza e successivamente vengono ristretti gli intervalli di valore) per itemset quantitativi (che nel nostro caso è una funzione che associa ogni attributo a un intervallo numerico del tipo [b, e]), quindi applicarlo ad un dataset (che nel nostro caso è quello della qualità dell'aria), per poi estrarre e ordinare le regole di associazione in base a 2 criteri, che sono:
1. il p-value -> misura quanto è probabile che una regola di associazione sia nata per caso -> con p-value > 0.1 la regola potrebbe essere casuale;
2. la lift -> misura quanto è più probabile osservare Y quando X è presente, rispetto al caso in cui X e Y siano indipendenti. La formula della lift è: Lift(X -> Y)= confidenza(X -> Y) / supporto(Y) -> se lift < 1 allora la presenza di X riduce la probabilità di Y.

In [11]:
# Innanzitutto scarico il dataset Air Quality
from ucimlrepo import fetch_ucirepo
dataset = fetch_ucirepo(id=360) # scarico e carico il dataset
dataset.data.features

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


Una volta che abbiamo caricato il dataset, dobbiamo prepararlo, ovverosia dobbiamo:
1. eliminare le colonne che non ci servono;
2. rinominare alcune colonne;
3. filtrare le righe con valori non-validi (-200 che rappresenta il missing value).

In [12]:
import polars as pl

aq = pl.from_pandas(dataset.data.features)
# Trasformo il dataset
aq = (
    aq
    .drop(["Date", "Time", "RH", "AH"]) # Elimino le colonne inutili
    .drop([col for col in aq.columns if col.endswith("(GT)")])  # Elimino le colonne che terminano con "(GT)" perchè sono le misure reali che non ci interessano
    .rename({   # Rinomino le colonne in modo tale che siano più comprensibili
        "PT08.S1(CO)": "CO",
        "PT08.S2(NMHC)": "NMHC",
        "PT08.S3(NOx)": "NOX",
        "PT08.S4(NO2)": "NOO",
        "PT08.S5(O3)": "OOO"
    })
    .filter(pl.any_horizontal(pl.all().ne(-200)))   # Elimino le righe con i valori non validi
)

aq  # Dataset aggiornato

CO,NMHC,NOX,NOO,OOO,T
i64,i64,i64,i64,i64,f64
1360,1046,1056,1692,1268,13.6
1292,955,1174,1559,972,13.3
1402,939,1140,1555,1074,11.9
1376,948,1092,1584,1203,11.0
1272,836,1205,1490,1110,11.2
…,…,…,…,…,…
1314,1101,539,1374,1729,21.9
1163,1027,604,1264,1269,24.3
1142,1063,603,1241,1092,26.9


Una volta che abbiamo aggiornato il dataset, dobbiamo codificare i valori continui del dataset, andando a trasformarli in interi discreti (quindi: 0, 1, 2, ..., n)-> questo lo facciamo, perchè l'algoritmo Apriori lavora su intervalli discreti.

In [13]:
# Creo un dizionario 'encoder', che ha come chiave una colonna del dataset (per esempio: CO, NOX) e ogni colonna ha una lista ordinata dei possibili valori
encoder = {
    col: sorted(aq.get_column(col).unique().to_list())
    for col in aq.columns
}
df = pl.DataFrame() # Nuovo dataframe per memorizzare i dati codificati

# Sostituisco i valori delle colonne con numeri interi
df = aq.select([
    pl.col(column).replace_strict(
        dict(zip(values, range(len(values))))  # Mappa i valori unici in numeri da 0 a N-1
    ).alias(column)
    for column, values in encoder.items()
])

df

CO,NMHC,NOX,NOO,OOO,T
i64,i64,i64,i64,i64,i64
665,622,699,985,976,145
597,531,817,852,680,142
707,515,783,848,782,128
681,524,735,877,911,119
577,412,847,783,818,121
…,…,…,…,…,…
619,677,182,667,1420,228
468,603,247,557,977,252
447,639,246,534,800,278


Arriviamo al primo punto vero e proprio dell'esercizio (1.A), in cui si implementa l'algoritmo Apriori quantitativo e lo si testa attraverso un test set.

In [16]:
from frozendict import frozendict

# Definisco una funzione per contare quante righe del dataset 'df' soddisfano tutte le condizioni dell'itemset, ovverosia per ogni attributo 'Ai' controllo che il valore è nell'intervallo [b, e]
def get_itemset_freq(itemset):
    conditions = [pl.col(col).is_between(rng[0], rng[1]) for col, rng in itemset.items()]   #  Verifico se il valore è dentro l'intervallo
    return len(df.filter(pl.reduce(lambda x, y: x & y, conditions))) # metto le condizioni in AND

# Calcolo il supporto di un itemset -> supporto(itemset) = (righe che soddisfano l'itemset) / (tutte le righe del dataset)
def get_support(itemset):
    return get_itemset_freq(itemset) / len(df)

# Implementazione dell'algoritmo Apriori
def apriori(min_support, min_step):
    # Partiamo con gli intervalli più ampi possibili
    SWk = { frozendict({k:(0, len(v) - 1) for k,v in encoder.items()}) }    # SWk = insieme iniziale degli itemset supportati
    res = pl.DataFrame()    # contiene gli itemset frequenti trovati
    step = 0

    while SWk:  # continuo a ciclare finchè ci sono itemset supportati
        step += 1
        print(f"Passo {step}: trovato {len(SWk)} itemsets nel passo precedente")

        # Genera nuovi itemset riducendo gli intervalli
        Wk = set()
        for itemset in SWk:
            for col in itemset:
                if itemset[col][1] - itemset[col][0] > min_step[col]:
                    Wk.add(itemset.set(col, (itemset[col][0] + min_step[col], itemset[col][1])))
                    Wk.add(itemset.set(col, (itemset[col][0], itemset[col][1] - min_step[col])))

        # Per ogni candidato in Wk, allora calcoliamo il supporto e se supera la soglia minima (del supporto) allora il candidato lo aggiugiamo a 'SWk_new'
        SWk_new = set()
        R = []
        for itemset in Wk:
            if (support := get_support(itemset)) >= min_support:
                SWk_new.add(itemset)
                R.append({'support': support} | {k: v for k, v in itemset.items() if v[0] != 0 or v[1] != len(encoder[k]) - 1})

        # Salvo i risultati e aggiorno SWk per il ciclo successivo
        SWk = SWk_new
        res = pl.concat([pl.DataFrame(R), res], how="diagonal")

    return res

# Eseguo l'algoritmo Apriori con supporto minimo 0.1 e riduzione a passi di 1/4 della dimensione originale
frequent_itemsets = apriori(0.1, {k: (len(v) // 4) for k,v in encoder.items()}) # NB!! Avevo usato altri valori per ridurre, solamente che ad esempio con 1 l'esecuzione ci metteva troppo tempo, mentre con 1/2 perdevo troppi itemset. Quindi ho deciso di utilizzare 1/4

Passo 1: trovato 1 itemsets nel passo precedente
Passo 2: trovato 12 itemsets nel passo precedente
Passo 3: trovato 78 itemsets nel passo precedente
Passo 4: trovato 354 itemsets nel passo precedente
Passo 5: trovato 1203 itemsets nel passo precedente
Passo 6: trovato 3174 itemsets nel passo precedente
Passo 7: trovato 6635 itemsets nel passo precedente
Passo 8: trovato 11137 itemsets nel passo precedente
Passo 9: trovato 15107 itemsets nel passo precedente
Passo 10: trovato 16524 itemsets nel passo precedente
Passo 11: trovato 14412 itemsets nel passo precedente
Passo 12: trovato 9826 itemsets nel passo precedente
Passo 13: trovato 5059 itemsets nel passo precedente
Passo 14: trovato 1847 itemsets nel passo precedente
Passo 15: trovato 417 itemsets nel passo precedente
Passo 16: trovato 45 itemsets nel passo precedente
Passo 17: trovato 1 itemsets nel passo precedente


In [17]:
(
    frequent_itemsets
    .sort(by="support") #Ordino gli itemset frequenti in base al supporto.
)

support,CO,NMHC,NOX,NOO,OOO,T
f64,list[i64],list[i64],list[i64],list[i64],list[i64],list[i64]
0.1001,"[260, 520]","[0, 933]","[305, 610]","[800, 1202]","[435, 872]","[109, 326]"
0.1001,"[0, 520]","[311, 622]","[305, 915]","[400, 802]","[435, 872]","[109, 435]"
0.1001,"[260, 520]","[311, 622]","[0, 915]","[400, 802]","[435, 872]","[0, 326]"
0.1001,"[260, 520]",null,"[305, 610]","[800, 1202]","[435, 872]","[109, 326]"
0.1001,"[260, 780]","[622, 933]","[0, 915]","[800, 1602]","[870, 1307]","[109, 326]"
…,…,…,…,…,…,…
0.907908,null,null,null,"[0, 1202]",null,null
0.918696,null,null,null,null,null,"[0, 326]"
0.927705,null,"[0, 933]",null,null,null,null


Passiamo al 3° punto dell'esercizio (1.C) in cui dobbiamo estrarre le regole di associazione a partire dagli itemset frequenti ottenuti con l'algoritmo Apriori. Inoltre, per ciascuna regola, dobbiamo calcolare:
1. supporto;
2. confidenza;
3. lift;
4. p-value.

In [19]:
from itertools import combinations
import polars as pl
import math
from frozendict import frozendict

def get_p_value(X, Y):  # Calcola quanto è probabile che la regola X -> Y (se X è presente, allora probabilmente anche Y è presente) si verifichi per caso
    N = len(df)  # Numero totale di righe nel dataset
    Nxy = get_itemset_freq(X | Y)  # Frequenza congiunta di X e Y
    Cx = get_itemset_freq(X)  # Frequenza di X
    Cy = get_itemset_freq(Y)  # Frequenza di Y

    # Numero di combinazioni possibili di Cx elementi tra N
    bin_X_Cx = math.comb(N, Cx)

    # Calcolo del p-value come probabilità cumulativa della distribuzione ipergeometrica
    p_value = 1 - sum(
        math.comb(Cy, k) * math.comb(N - Cy, Cx - k) / bin_X_Cx
        for k in range(Nxy)
    )
    return p_value


def mine_rules(frequent_itemsets, min_confidence, min_lift=1):  # Estraggo tutte le regole di associazione X -> Y dagli itemset frequenti, filtrando in base alla confidenza, lift e p-value
    association_rules = []
    for itemset in frequent_itemsets.rows(named=True):  # ciclo sugli itemset
        itemset = {k: v for k, v in itemset.items() if v}  # Rimuove valori nulli
        support = itemset.pop("support")  # Estrae il supporto dell'itemset

        # Se l'itemset ha meno di 2 elementi, non può generare regole
        if len(itemset) < 2:
            continue

        # Genero tutte le possibili combinazioni di sottoinsiemi X -> capiamo, allora, come è formata la X e la Y
        for i in range(1, len(itemset)):
            for combination in combinations(itemset.items(), i):
                # Per esempio: itemset = {"CO": (1,4), "NOX": (3,6)} allora: X = {"CO": (1,4)} e Y = {"NOX": (3,6)}
                X = frozendict(combination)  # Antecedente della regola
                Y = frozendict({k: v for k, v in itemset.items() if k not in X})  # Consequente

                # Calcolo il supporto
                support_X = get_support(X)
                support_Y = get_support(Y)

                # Evito divisioni per zero --> se non lo metto mi dava errore
                if support_X == 0 or support_Y == 0:
                    continue

                # Calcolo la 'confidenza' e la 'lift'
                confidence = support / support_X
                lift = confidence / support_Y

                # Controllo i vincoli minimi
                if confidence < min_confidence or lift <= min_lift:
                    continue

                # Calcolo il 'p-value'
                p_value = get_p_value(X, Y)

                # Costruisco la regola
                rule = {
                    "X": list(X.keys()),
                    "Y": list(Y.keys()),
                    "support": support,
                    "confidence": confidence,
                    "lift": lift,
                    "p_value": p_value
                } | itemset  # Mantiene i dettagli dell'itemset originale

                association_rules.append(rule)

    # Restituisce le regole
    return pl.DataFrame(association_rules)

# Generazione delle regole di associazione con i vincoli minimi richiesti
association_rules = mine_rules(frequent_itemsets, min_confidence=0.99, min_lift=1.54)   # Ho scelto questi valori di confidence e di lift, perchè avevo provato con altri valori, ma dopo molte ore non avevo ancora ottenuto un risultato

association_rules

X,Y,support,confidence,lift,p_value,CO,NMHC,NOX,NOO,OOO,T
list[str],list[str],f64,f64,f64,f64,list[i64],list[i64],list[i64],list[i64],list[i64],list[i64]
"[""CO"", ""NOX"", … ""OOO""]","[""NMHC"", ""T""]",0.12924,0.997425,1.732248,-4.4409e-16,"[260, 520]","[311, 933]","[305, 610]","[800, 1202]","[435, 872]","[109, 435]"
"[""CO"", ""NMHC"", … ""T""]","[""NOO""]",0.108108,0.991837,1.774648,-4.4409e-16,"[0, 260]","[0, 311]","[610, 1220]","[0, 802]","[0, 437]","[0, 217]"
"[""CO"", ""NOX"", … ""T""]","[""NMHC"", ""OOO""]",0.122901,0.992812,1.557832,0.0,"[260, 520]","[311, 933]","[305, 610]","[800, 1202]","[0, 1307]","[218, 435]"
"[""NOX"", ""NOO"", ""T""]","[""CO"", ""NMHC"", ""OOO""]",0.105439,0.993711,1.659753,0.0,"[0, 520]","[0, 622]","[610, 915]","[400, 802]","[0, 872]","[109, 326]"
"[""CO"", ""NOX"", … ""T""]","[""NMHC"", ""OOO""]",0.105439,0.995798,1.619906,3.3307e-16,"[0, 520]","[0, 622]","[610, 915]","[400, 802]","[0, 872]","[109, 326]"
…,…,…,…,…,…,…,…,…,…,…,…
"[""NMHC""]","[""NOX"", ""OOO""]",0.250028,0.99469,1.541941,-2.2204e-16,null,"[0, 311]","[305, 1220]",null,"[0, 872]",null
"[""NOX""]","[""CO"", ""NMHC"", ""OOO""]",0.256034,0.999566,1.570896,1.1102e-16,"[260, 1040]","[311, 1244]","[0, 305]",null,"[435, 1742]",null
"[""NMHC""]","[""CO"", ""NOX"", ""NOO""]",0.3221,0.993823,1.541926,0.0,"[260, 1040]","[622, 1244]","[0, 915]","[400, 1602]",null,null


In [20]:
association_rules.sort(by='p_value')    # ordino le regole di associazione in base al valore del p-value in ordine crescente

X,Y,support,confidence,lift,p_value,CO,NMHC,NOX,NOO,OOO,T
list[str],list[str],f64,f64,f64,f64,list[i64],list[i64],list[i64],list[i64],list[i64],list[i64]
"[""CO"", ""NOX"", … ""T""]","[""NMHC"", ""OOO""]",0.126237,1.0,1.626741,-1.1102e-15,"[0, 520]","[0, 622]","[610, 915]","[0, 802]","[0, 872]","[0, 217]"
"[""CO"", ""NOO"", … ""T""]","[""NMHC"", ""NOX""]",0.166389,1.0,1.573228,-1.1102e-15,"[0, 260]","[0, 622]","[305, 1220]","[400, 1202]","[0, 872]","[0, 326]"
"[""CO"", ""NOO"", … ""T""]","[""NMHC"", ""NOX""]",0.166389,1.0,1.573228,-1.1102e-15,"[0, 260]","[0, 622]","[305, 1220]","[400, 1602]","[0, 872]","[0, 326]"
"[""CO"", ""NOO"", … ""T""]","[""NMHC"", ""NOX""]",0.166389,1.0,1.573228,-1.1102e-15,"[0, 260]","[0, 622]","[305, 1220]","[400, 1202]","[0, 1307]","[0, 326]"
"[""CO"", ""NOO"", … ""T""]","[""NMHC"", ""NOX""]",0.166389,1.0,1.573228,-1.1102e-15,"[0, 260]","[0, 622]","[305, 1220]","[400, 1602]","[0, 1307]","[0, 326]"
…,…,…,…,…,…,…,…,…,…,…,…
"[""CO"", ""NOX"", ""NOO""]","[""NMHC"", ""OOO""]",0.171616,0.990372,1.554003,1.1102e-15,"[260, 520]","[311, 933]","[305, 1220]","[800, 1202]","[0, 1307]",null
"[""NOX"", ""OOO"", ""T""]","[""CO"", ""NMHC"", ""NOO""]",0.263263,0.990377,1.59179,1.1102e-15,"[260, 1040]","[311, 1244]","[0, 915]","[400, 1602]","[870, 1742]","[109, 435]"
"[""NMHC"", ""NOX""]","[""CO"", ""NOO"", ""OOO""]",0.250028,0.99469,1.542739,1.2212e-15,"[0, 520]","[0, 311]","[305, 1220]","[0, 1202]","[0, 872]",null


In [21]:
association_rules.sort(by='lift', descending=True)  # ordino le regole di associazione in base al valore della lift in ordine decrescente

X,Y,support,confidence,lift,p_value,CO,NMHC,NOX,NOO,OOO,T
list[str],list[str],f64,f64,f64,f64,list[i64],list[i64],list[i64],list[i64],list[i64],list[i64]
"[""CO"", ""NMHC"", … ""T""]","[""NOO"", ""OOO""]",0.113669,0.994163,3.209524,4.4409e-16,"[520, 1040]","[622, 1244]","[0, 305]","[400, 1602]","[870, 1742]","[0, 217]"
"[""CO"", ""NOX"", … ""OOO""]","[""NMHC""]",0.146035,0.996206,3.073745,0.0,"[520, 1040]","[622, 1244]","[0, 305]","[800, 1602]","[870, 1742]",null
"[""CO"", ""NOX"", … ""T""]","[""NMHC""]",0.138249,0.995994,3.073088,2.2204e-16,"[520, 1040]","[622, 1244]","[0, 305]","[800, 1602]","[870, 1742]","[0, 326]"
"[""CO"", ""NOX"", … ""T""]","[""NMHC""]",0.135802,0.995922,3.072866,9.9920e-16,"[520, 1040]","[622, 1244]","[0, 305]","[800, 1602]","[870, 1742]","[109, 435]"
"[""CO"", ""NOX"", … ""T""]","[""NMHC""]",0.128017,0.995675,3.072104,0.0,"[520, 1040]","[622, 1244]","[0, 305]","[800, 1602]","[870, 1742]","[109, 326]"
…,…,…,…,…,…,…,…,…,…,…,…
"[""NMHC"", ""OOO"", ""T""]","[""CO"", ""NOX"", ""NOO""]",0.164498,0.992617,1.540056,-6.6613e-16,"[260, 1040]","[622, 933]","[0, 915]","[400, 1602]","[870, 1742]","[109, 435]"
"[""NMHC"", ""OOO"", ""T""]","[""CO"", ""NOX"", ""NOO""]",0.194083,0.992605,1.540037,-4.4409e-16,"[260, 1040]","[622, 933]","[0, 915]","[400, 1602]","[435, 1742]","[109, 326]"
"[""NMHC"", ""T""]","[""CO"", ""NOX"", ""NOO""]",0.194083,0.992605,1.540037,-4.4409e-16,"[260, 1040]","[622, 933]","[0, 915]","[400, 1602]",null,"[109, 326]"
